# Ejercicio 8 activos que cotizan en cada indice Yahoo

- Obtén los activos que cotizan en cada índice y en qué divisa están cotizando
- Recomendación: Yahoo Finance 
- Tiempo objetivo: 45 minutos
- Los componentes de cada índice se renuevan cada x meses. La mejora manera de descargarnos datos sobre un índice no es consultar una lista fija de activos, sino consultar previamente qué activos están #### cotizando en cada índice antes de hacer la consulta activo por activo.

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

#https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe#/

In [2]:
tickers_de_indices=("%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", 
                    "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI")

Mediante esta funcion se obtiene un dataframe de los indices, los componentes de los mismos y sus divisas, eliminado aquellos que no tienen posicion. En aquellos casos en los que el proceso falle, bien sea porque no hay ningun dato, o los datos tienen algo raro que los pueda hacer inconsistentes, se informa del error, y se continua. 

In [3]:
def componentes_indice(tickers_de_indices):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    n=0
    for i in tickers_de_indices:

        Index=i
        
        #primero se obtiene la divisa
        url="https://es.finance.yahoo.com/quote/"+i+"/components/"
        soup  = requests.get(url)
        soup  = BeautifulSoup(soup.content, 'html.parser')
        name_box = soup.find('span', attrs={'data-reactid': '4'})
        
        try:
            name = name_box.text.strip()
            divisa=name[len(name)-3:len(name)]
            print(i,divisa)
            
            # Reconstruimos la tabla. Tiene 6 columnas (símbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
            # Nos interesa obtener el símbolo y el nombre de la empresa.
            # Ojo, queremos importar únicamente los activos que tengan cotización. 
            
            soup  = requests.get(url)
            soup  = BeautifulSoup(soup.content, 'html.parser')
            
            name_box = soup.find_all('tr')
            name_boxccc=name_box[1]
            name_boxccc = name_boxccc.get_text(strip=True)
            lista_indice=[]
            lista_activo=[]
            lista_divisa=[]
            lista_num_errores=[]
            lista_empresa=[]
            lista_resto=[]

            for i in range(1,len(name_box)-1):
                try:
                    lista_indice.append(Index)
                    name_boxccc=name_box[i]
                    name_boxccc = name_boxccc.get_text(strip=True)
                    name_boxccc=name_boxccc.split(sep='.')
                    lista_activo.append(name_boxccc[0]+'.'+name_boxccc[1][0:2])
                    lista_empresa.append(name_boxccc[1][2:5]+'.'+name_boxccc[2][0:len(name_boxccc[2])])
                    lista_resto.append(name_boxccc)
                    lista_divisa.append(divisa)

                except:
                    lista_num_errores.append(i)

            if len(lista_activo)==len(lista_divisa) & len(lista_activo)==len(lista_empresa) & len(lista_activo)==len(lista_resto):

                df=pd.DataFrame ({'indice':lista_indice,
                                'activo':lista_activo,
                                'divisa':lista_divisa,
                                'resto':lista_resto})
                df['posicion']=np.where(df['resto'].astype(str).str.contains('%', regex=False)==True,1,0)

                df=df[df['posicion']==1]
                df=df.drop(['resto','posicion'],1)

                if n==0:
                    dfacum=df
                else:
                    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
                    dfacum.reset_index(drop=True)
                n=n+1

            else:
                print('El activo '+str(i)+'no se ha podido incorporar por inconsistencias en los datos importados')

        except:
            
            print('Para la referencia '+str(i)+' no se ha podido descargar la informacion')
    
    return dfacum
    

In [4]:
dfacum=componentes_indice(tickers_de_indices)
dfacum

%5EBFX EUR
%5EBVSP BRL
El activo 30no se ha podido incorporar por inconsistencias en los datos importados
%5EDJI USD
%5EFCHI EUR
%5EFTSE GBP
%5EGDAXI EUR
%5EHSI HKD
%5EIBEX EUR
%5EMXX MXN
El activo 30no se ha podido incorporar por inconsistencias en los datos importados
Para la referencia %5EJKSE no se ha podido descargar la informacion
%5EMERV  en
%5EOMXSPI SEK
%5EOSEAX NOK
El activo 30no se ha podido incorporar por inconsistencias en los datos importados
%5ESSMI CHF
%5ESTI SGD


,activo,divisa,indice
3,UCB.BR,EUR,%5EBFX
9,KBC.BR,EUR,%5EBFX
13,COFB.BR,EUR,%5EBFX
14,ONTEX.BR,EUR,%5EBFX
0,"DISThe Walt Disney Company111,46-0,06-0,05%8.93",USD,%5EDJI
1,"WBAWalgreens Boots Alliance, Inc.42",USD,%5EDJI
2,PFEPfizer Inc.32,USD,%5EDJI
3,"MMM3M Company155,930,36+0,23%2.07",USD,%5EDJI
4,"RTXRaytheon Technologies Corporation61,64-0,28...",USD,%5EDJI
5,DOWDow Inc.40,USD,%5EDJI


### Otra manera de hacerlo

In [1]:
import urllib.request
import numpy as np
import pandas as pd

In [2]:
tickers_de_indices=["%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", 
                    "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"]

In [3]:
for indice in tickers_de_indices:
    
    url="https://es.finance.yahoo.com/quote/"+indice+"/components/"
    
    #Extraemos la divisa en la que cotizan los activos.
    html=str(urllib.request.urlopen(url).read())
    posicion=html.find('Divisa en')
    divisa=html[posicion:posicion+13].split()[2]
    
    #Leo tabla de indices
    dfs = pd.read_html(url)
    dfs=dfs[0]
    
    #Comprobamos si Yahoo publica algún activo del índice
    if not dfs.empty:
        
        #Nos quedamos únicamente los activos que tengan cotización. 
        nombre_empresas=dfs.dropna().iloc[:,[0,1]].reset_index(drop=True)
        print("Indice "+indice.replace("%5", "")+ " en "+divisa)
        print("")
        print(nombre_empresas)
        print("")
        
    else:
        
        print("El indice no tiene activos descargables")

Indice EBFX en EUR

    Símbolo Nombre de la empresa
0    UCB.BR               UCB SA
1    KBC.BR         KBC Group NV
2   COFB.BR         Cofinimmo SA
3  ONTEX.BR       Ontex Group NV

Indice EBVSP en BRL

      Símbolo                               Nombre de la empresa
0    PCAR4.SA               Companhia Brasileira de Distribuicao
1    USIM5.SA           Usinas Siderurgicas de Minas Gerais S.A.
2    OIBR4.SA                                            Oi S.A.
3    JBSS3.SA                                           JBS S.A.
4    LREN3.SA                                  Lojas Renner S.A.
5    SBSP3.SA  Companhia de Saneamento Basico do Estado de Sa...
6    CMIG4.SA               Companhia Energetica de Minas Gerais
7   KLBN11.SA                                  KLABIN S/A UNT N2
8    EQTL3.SA                            Equatorial Energia S.A.
9    BBDC4.SA                                Banco Bradesco S.A.
10   ITSA4.SA                   Itausa - Investimentos Itau S.A.
11   BBDC3.SA

### Otra manera de hacerlo

In [ ]:
from requests_html import HTMLSession
import pandas as pd

In [ ]:
tickers_de_indices = [
    "%5EBFX", "%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE",
    "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE",
    "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"
]

In [ ]:
for indice in tickers_de_indices:

    # Construimos la url que contiene la información de los activos de cada índice.
    web = f"https://es.finance.yahoo.com/quote/{indice}/components/"
    datos_web = HTMLSession().get(web).html

    # activos del indice
    activos = pd.read_html(datos_web.find('table', first=True).html)[0]

    if len(activos) >= 1:  # Comprobamos si Yahoo publica algún activo del índice.

        # Ojo, queremos importar únicamente los activos que tengan cotización.
        activos_con_datos = activos.dropna(subset=['Último precio'])

        # simbolo de cada empresa del índice
        activos_a_importar = activos_con_datos['Símbolo'].to_list()

        # nombre de cada empresa del índice
        nombres_empresas = activos_con_datos['Nombre de la empresa'].to_list()

        # divisa de las empresas del índice
        divisa_empresas = (datos_web
                           .find('#Col1-0-Components-Proxy', first=True)
                           .find('span', first=True)
                           .text
                           .split(' ')[-1])
    else:
        print(f"El índice {indice} no tiene activos descargables")

    print(nombres_empresas)
    print(divisa_empresas)

### Otra manera de hacerlo

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
indices = ["%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"]

for i in indices:
    
    empresas=[]
    activos_a_importar=[]
    url = f'https://es.finance.yahoo.com/quote/{i}/components?p={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    texto=soup.find("div")
    sig=texto.findAll("span")
    linea=str(sig[7])
    divisa=linea[(linea.find("Divisa")+10):linea.find("</")]

    url = f'https://es.finance.yahoo.com/quote/{i}/components?p={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tabla=soup.find('table')
    trs=tabla.findAll("tr")    
    for tr in trs[1:]:
        tds=tr.findAll('td')
        a=str(tds[1])
        b=str(tds[2])
        c=str(tds[0])
        nombre=a[(a.find('>')+1):(a.find('</'))]
        cotizacion=b[(b.find('>')+1):(b.find('</'))]
        c=c[(c.find("title")+7):]
        activo=c[:(c.find('">'))]
        if cotizacion != "" and cotizacion != "0,00":            
            empresas.append(nombre)
            activos_a_importar.append(activo)
    
    print(f'Las empresas del indice {i} que tiene cotización, con divisa {divisa}, son las siguientes')
    print(empresas)
    print("")

Las empresas del indice %5EBFX que tiene cotización, con divisa EUR, son las siguientes
['UCB SA', 'KBC Group NV', 'Cofinimmo SA', 'Ontex Group NV']

Las empresas del indice %5EBVSP que tiene cotización, con divisa BRL, son las siguientes
['Companhia Brasileira de Distribuicao', 'EDP - Energias do Brasil S.A.', 'Companhia Energetica de Minas Gerais', 'JBS S.A.', 'Ultrapar Participacoes S.A.', 'Oi S.A.', 'Equatorial Energia S.A.', 'TIM Participacoes S.A.', 'EcoRodovias Infraestrutura e Logistica S.A.', 'Multiplan Empreendimentos Imobiliarios S.A.', 'Braskem S.A.', 'BB Seguridade Participacoes S.A.', 'BR Malls Participacoes S.A.', 'Lojas Renner S.A.', 'Telefonica Brasil S.A.', 'Petroleo Brasileiro S.A. - Petrobras', 'Petroleo Brasileiro S.A. - Petrobras', 'Companhia de Saneamento Basico do Estado de Sao Paulo - SABESP', 'CCR S.A.', 'Raia Drogasil S.A.', 'KLABIN S/A  UNT     N2', 'Usinas Siderurgicas de Minas Gerais S.A.', 'Banco do Brasil S.A.', 'Banco Bradesco S.A.', 'Lojas Americanas S

### Otra manera de hacerlo

In [ ]:
import requests
from lxml import html
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
tqdm.pandas()

In [ ]:
indices = ["%5EBFX","%5EBVSP", "%5EDJI",
           "%5EFCHI", "%5EFTSE", "%5EGDAXI",
           "%5EHSI", "%5EIBEX", "%5EMXX",
           "%5EJKSE", "%5EMERV", "%5EOMXSPI",
           "%5EOSEAX", "%5ESSMI", "%5ESTI"] 

In [ ]:
def getComponents(index:str):
    
    print(f'Downloading index {index}')
    
    result = []
    url = f"https://finance.yahoo.com/quote/{index}/components?p={index}"
    s = requests.Session()
    retries = Retry(total=3, backoff_factor=2, status_forcelist=[ 502, 503, 504 ])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    website_url = s.get(url)
    
    tree = html.fromstring(website_url.content)
    currency = tree.xpath('//*[@id="quote-header-info"]/div[2]/div[1]/div[2]/span')[0].text
    result.append({'currency': currency[-3:]})
    
    soup = BeautifulSoup(website_url.text)
    table = soup.find('table', attrs={'class':"W(100%) M(0) BdB Bdc($finLightGray)"})
    try:
        tableData = table.tbody.find_all("tr")
        for register in tableData:
            symbol = register.find_all("td")[0].text
            name = register.find_all("td")[1].text
            volume = register.find_all("td")[3].text
            if len(volume) > 0:
                result.append({symbol: {'name': name}})
    
    except Exception as e:
        print(f'Component {index} could not be downloaded.')
        print(e)

    return result 

In [ ]:
dictComponents = {index:getComponents(index) for index in tqdm(indices)}

In [ ]:
dictComponents[indices[1]]